In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow import keras

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import accuracy_score
from skimage import io

In [4]:
# Load the LFW dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [5]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(lfw_people.images, lfw_people.target, test_size=0.25, random_state=42)


In [6]:
# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)


In [7]:
# Normalize and preprocess images
X_train = X_train / 255.0
X_test = X_test / 255.0


In [8]:
# Build a CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 37, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Reshape image data for CNN input
X_train = X_train.reshape(-1, 50, 37, 1)
X_test = X_test.reshape(-1, 50, 37, 1)

In [10]:

# Train the model
model.fit(X_train, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
13/13 [==============================] - 4s 146ms/step - loss: 1.7990 - accuracy: 0.3847 - val_loss: 1.6610 - val_accuracy: 0.4124
Epoch 2/10
13/13 [==============================] - 2s 116ms/step - loss: 1.7389 - accuracy: 0.3938 - val_loss: 1.6485 - val_accuracy: 0.4124
Epoch 3/10
13/13 [==============================] - 2s 121ms/step - loss: 1.7203 - accuracy: 0.3938 - val_loss: 1.6249 - val_accuracy: 0.4124
Epoch 4/10
13/13 [==============================] - 1s 111ms/step - loss: 1.6931 - accuracy: 0.3951 - val_loss: 1.5929 - val_accuracy: 0.4124
Epoch 5/10
13/13 [==============================] - 1s 114ms/step - loss: 1.6690 - accuracy: 0.3990 - val_loss: 1.5237 - val_accuracy: 0.4175
Epoch 6/10
13/13 [==============================] - 1s 106ms/step - loss: 1.6002 - accuracy: 0.4301 - val_loss: 1.4658 - val_accuracy: 0.4278
Epoch 7/10
13/13 [==============================] - 1s 108ms/step - loss: 1.5080 - accuracy: 0.4585 - val_loss: 1.3804 - val_accuracy: 0.5258
Epoch 

In [11]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_pred_classes)

print(f"Accuracy: {accuracy * 100:.2f}%")


11/11 [==============================] - 0s 15ms/step
Accuracy: 61.80%


In [12]:
import cv2


In [14]:
class_label_to_name = {
    0: "Face not detected",
    1: "Emma watson",
    # Add more mappings as needed
}

def recognize_face(image_path):
    img = io.imread(image_path, as_gray=True)
    img = img / 255.0
    img = cv2.resize(img, (37, 50))
    img = img.reshape(1, 50, 37, 1)  # Reshape for CNN input
    predicted_label = model.predict(img)
    predicted_class = np.argmax(predicted_label)
    recognized_person = class_label_to_name.get(predicted_class, "Unknown")  # Default to "Unknown" if label not found
    return recognized_person

# Example usage:
test_image_path = "C:/Users/Jeyaa N.K.S/Downloads/Emma Watson.jpg"  
recognized_person = recognize_face(test_image_path)
print(f"Recognized Person: {recognized_person}")

1/1 [==============================] - 0s 63ms/step
Recognized Person: Emma watson
